In [205]:
import pandas as pd
func_tag_dic = {"Data Analysis":0, "Data Visualization":1 , "Data Preparation":2}

In [206]:
func_corpura = pd.read_csv("function.csv")

In [207]:
func_corpura

,URL,contributor,label,excerpt
0,https://github.com/KnowledgeCaptureAndDiscover...,Pratheek,Data Preparation,If you want to classify your videos or extract...
1,https://github.com/driving-behavior/DBNet,Pratheek,Data Preparation,DBNet is a large-scale driving behavior datase...
2,https://github.com/hezhangsprinter/DCPDN,Pratheek,Data Preparation,We propose a new end-to-end single image dehaz...
3,https://github.com/hezhangsprinter/DID-MDN,Pratheek,Data Analysis,We present a novel density-aware multi-stream ...
4,https://github.com/foolwood/DaSiamRPN,Pratheek,Data Visualization,SiamRPN formulates the task of visual tracking...
...,...,...,...,...
86,https://github.com/CMU-Perceptual-Computing-La...,Ling Li,Data Analysis,2D real-time multi-person keypoint detection:\...
87,https://github.com/airbnb/airpal,Yi Xie,Data Visualization,"Airpal is a web-based, query execution tool wh..."
88,https://github.com/HumbleSoftware/envisionjs,Yi Xie,Data Visualization,Fast interactive HTML5 charts.
89,https://github.com/apache/incubator-echarts,Yi Xie,Data Visualization,"Apache ECharts (incubating) is a free, powerfu..."


 ## Distribution

In [208]:
count = func_corpura.groupby('label').count()
count

,URL,contributor,excerpt
label,,,
Data Analysis,33,33,33
Data Preparation,28,28,28
Data Visualization,30,30,30


In [209]:
for tag in func_tag_dic:
    func_corpura.replace(tag,func_tag_dic[tag], inplace=True)
import nltk
from nltk.corpus import stopwords
#Encounter some errors when we downloaded treebank, so use ssl
import ssl
try:
    _create_unverified_https_context=ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context
nltk.download('stopwords')
#----------Download End-------------------

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import pandas as pd
from sklearn.metrics import precision_score,recall_score,roc_curve, auc, precision_recall_curve, average_precision_score
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from sklearn.naive_bayes import MultinomialNB
stop_words = stopwords.words('english')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [210]:
X_func, Y_func = func_corpura["excerpt"], func_corpura["label"]

## CountVectorizer + LogisticRegression

In [247]:
pipeline = make_pipeline(CountVectorizer(), LogisticRegression(penalty='l2',multi_class='multinomial',solver = 'lbfgs'))
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)

def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score

pipeline.fit(X_train, Y_train)
y_pred_class = pipeline.predict(X_test)
y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')
print(y_pred_class)

              precision    recall  f1-score   support

           0       0.38      0.86      0.52         7
           1       0.67      0.29      0.40         7
           2       0.50      0.22      0.31         9

    accuracy                           0.43        23
   macro avg       0.51      0.46      0.41        23
weighted avg       0.51      0.43      0.40        23

null accuracy: 39.13%
accuracy score: 43.48%
model is 4.35% more accurate than null accuracy


accuracy score: 43.48%
Precision score : 0.5138888888888888
Recall score  : 0.45502645502645506


[0 0 1 0 2 0 0 2 1 0 0 0 2 0 0 0 0 0 0 2 1 0 0]


In [94]:

cv = StratifiedKFold(n_splits = 5, shuffle=True)

cv_results = cross_validate(pipeline, X_func, Y_func, cv=5,scoring='accuracy')
print(cv_results['test_score'])
print('Mean accuracy of CV:',cv_results['test_score'].mean())

tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)
    
i = 0
print('Description ROC')

[0.63157895 0.36842105 0.57894737 0.47058824 0.41176471]
Mean accuracy of CV: 0.49226006191950467
Description ROC


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


## 2) TfidfVectorizer + LogisticRegression


In [246]:

pipeline = make_pipeline(TfidfVectorizer(), LogisticRegression(solver = 'liblinear'))
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')
print(y_pred_class)

              precision    recall  f1-score   support

           0       0.32      1.00      0.48         6
           1       1.00      0.22      0.36         9
           2       1.00      0.25      0.40         8

    accuracy                           0.43        23
   macro avg       0.77      0.49      0.41        23
weighted avg       0.82      0.43      0.41        23

null accuracy: 39.13%
accuracy score: 43.48%
model is 4.35% more accurate than null accuracy


accuracy score: 43.48%
Precision score : 0.7719298245614036
Recall score  : 0.49074074074074076


[1 0 0 0 0 0 1 0 0 0 0 0 0 0 2 0 2 0 0 0 0 0 0]


## 3) TfidfVectorizer + NaiveBayes¶


In [229]:

pipeline = make_pipeline(TfidfVectorizer(), MultinomialNB())
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')


              precision    recall  f1-score   support

           0       0.19      1.00      0.32         4
           1       1.00      0.11      0.20         9
           2       0.00      0.00      0.00        10

    accuracy                           0.22        23
   macro avg       0.40      0.37      0.17        23
weighted avg       0.42      0.22      0.13        23

null accuracy: 43.48%
accuracy score: 21.74%
model is 21.74% less accurate than null accuracy


accuracy score: 21.74%
Precision score : 0.3968253968253968
Recall score  : 0.3703703703703704




## 4) TfidfVectorizer + Perceptron¶


In [238]:

pipeline = make_pipeline(TfidfVectorizer(), Perceptron())
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
#y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')

              precision    recall  f1-score   support

           0       0.40      0.80      0.53         5
           1       0.50      0.33      0.40         9
           2       0.29      0.22      0.25         9

    accuracy                           0.39        23
   macro avg       0.40      0.45      0.39        23
weighted avg       0.39      0.39      0.37        23

null accuracy: 39.13%
accuracy score: 39.13%
model is exactly as accurate as null accuracy


accuracy score: 39.13%
Precision score : 0.3952380952380952
Recall score  : 0.4518518518518519




## K neighboors

In [242]:
from sklearn.neighbors import KNeighborsClassifier

pipeline = make_pipeline(TfidfVectorizer(), KNeighborsClassifier(n_neighbors=3))
X_train, X_test, Y_train, Y_test = train_test_split(X_func, Y_func)


def display_accuracy_score(y_test, y_pred_class):
    score = accuracy_score(y_test, y_pred_class)
    print('accuracy score: %s' % '{:.2%}'.format(score))
    return score

def display_null_accuracy(y_test):
    value_counts = pd.value_counts(y_test)
    null_accuracy = max(value_counts) / float(len(y_test))
    print('null accuracy: %s' % '{:.2%}'.format(null_accuracy))
    return null_accuracy

def display_accuracy_difference(y_test, y_pred_class):
    null_accuracy = display_null_accuracy(y_test)
    accuracy_score = display_accuracy_score(y_test, y_pred_class)
    difference = accuracy_score - null_accuracy
    if difference > 0:
        print('model is %s more accurate than null accuracy' % '{:.2%}'.format(difference))
    elif difference < 0:
        print('model is %s less accurate than null accuracy' % '{:.2%}'.format(abs(difference)))
    elif difference == 0:
        print('model is exactly as accurate as null accuracy')
    return null_accuracy, accuracy_score


pipeline.fit(X_train, Y_train)

y_pred_class = pipeline.predict(X_test)
#y_pred_vals = pipeline.predict_proba(X_test)
results_df = pd.DataFrame({"x_test": X_test,  "y_TF_pred": y_pred_class, "y_actual": Y_test})
#print(results_df)
#print(confusion_matrix(y_test, y_pred_class))
#print('-' * 75 + '\nClassification Report\n')
print(classification_report(Y_test, y_pred_class))

display_accuracy_difference(Y_test, y_pred_class)
print('\n')
display_accuracy_score(Y_test, y_pred_class)
print('Precision score :',precision_score(Y_test, y_pred_class,average='macro'))
print('Recall score  :',recall_score(Y_test, y_pred_class,average='macro'))

print('\n')


              precision    recall  f1-score   support

           0       0.40      0.67      0.50         9
           1       0.17      0.33      0.22         3
           2       0.50      0.09      0.15        11

    accuracy                           0.35        23
   macro avg       0.36      0.36      0.29        23
weighted avg       0.42      0.35      0.30        23

null accuracy: 47.83%
accuracy score: 34.78%
model is 13.04% less accurate than null accuracy


accuracy score: 34.78%
Precision score : 0.35555555555555557
Recall score  : 0.3636363636363636


